In [ ]:
# DONT RUNT THIS CELL
# moved to the beginning to not lost it in case we need it later


#drop columns i dont care or have too many empty values:
aux_training = aux_training.drop('date_time', axis=1)

aux_training = aux_training.drop('comp1_rate', axis=1)
aux_training = aux_training.drop('comp1_inv', axis=1)
aux_training = aux_training.drop('comp1_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp2_rate', axis=1)
aux_training = aux_training.drop('comp2_inv', axis=1)
aux_training = aux_training.drop('comp2_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp3_rate', axis=1)
aux_training = aux_training.drop('comp3_inv', axis=1)
aux_training = aux_training.drop('comp3_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp4_rate', axis=1)
aux_training = aux_training.drop('comp4_inv', axis=1)
aux_training = aux_training.drop('comp4_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp5_rate', axis=1)
aux_training = aux_training.drop('comp5_inv', axis=1)
aux_training = aux_training.drop('comp5_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp6_rate', axis=1)
aux_training = aux_training.drop('comp6_inv', axis=1)
aux_training = aux_training.drop('comp6_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp7_rate', axis=1)
aux_training = aux_training.drop('comp7_inv', axis=1)
aux_training = aux_training.drop('comp7_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp8_rate', axis=1)
aux_training = aux_training.drop('comp8_inv', axis=1)
aux_training = aux_training.drop('comp8_rate_percent_diff', axis=1)

aux_training = aux_training.drop('visitor_hist_starrating', axis=1)
aux_training = aux_training.drop('visitor_hist_adr_usd', axis=1)

aux_training = aux_training.drop('srch_query_affinity_score', axis=1)

aux_training = aux_training.drop('gross_bookings_usd', axis=1)

aux_training = aux_training.drop('prop_review_score', axis=1)
aux_training = aux_training.drop('prop_location_score2', axis=1)
aux_training = aux_training.drop('orig_destination_distance', axis=1)

In [ ]:
pip install lightgbm

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import gc
from tabulate import tabulate
from pathlib import Path
from cProfile import label
from jinja2 import Undefined
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics,tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from datamining_machine import DataMiningMachine
from sklearn.model_selection import train_test_split

In [29]:
test_data  = pd.read_csv("test_set_VU_DM.csv")

In [3]:
training_data  = pd.read_csv("training_set_VU_DM.csv")

In [ ]:
training_data.head()

In [4]:
#create new column for NDCG evaluation
training_data['target_col'] = training_data.apply(
    lambda row: (5 if row.booking_bool == 1 else (1 if row.click_bool == 1 else 0)), axis=1)

In [5]:
aux_training = copy.deepcopy(training_data)

In [ ]:
aux_training.head()

In [ ]:
print("len target 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len target 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len target 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))


In [ ]:
## get set balanced for target column
#np.random.seed(0)
#balanced = aux_training.groupby('target_col')
#balanced = balanced.apply(lambda x: x.sample(balanced.size().min()))

print("len target 0: ",len(balanced.loc[balanced["target_col"] == 0]))
print("len target 1: ",len(balanced.loc[balanced["target_col"] == 1]))
print("len target 5: ",len(balanced.loc[balanced["target_col"] == 5]))

In [ ]:
#aux_training = aux_training.sample(frac=.5) #random sample

print("len target 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len target 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len target 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))

In [6]:
aux_training = aux_training.drop('date_time', axis=1) # cause me dont care datee

In [7]:
##  drop columns that are not in test set (??? ) 
aux_training = aux_training.drop('click_bool', axis=1)   #replaced by target_col and not in test set 
aux_training = aux_training.drop('booking_bool', axis=1) #replaced by target_col and not in test set 

aux_training = aux_training.drop('position', axis=1)     # not in test set 
aux_training = aux_training.drop('gross_bookings_usd', axis=1) # not in test set 


In [ ]:
# CAN WE DROP THIS AFTER TRAINING?
#aux_training = aux_training.drop('position', axis=1)          # (not in test set)not sure about drop this one. Maybe some transformation ??
#aux_training = aux_training.drop('gross_booking_usd', axis=1) # (not in test set)not sure about drop this one. Maybe some transformation ??

In [ ]:
print("Empty values per column: \n",aux_training.isnull().sum())

In [8]:
feature = aux_training.drop('target_col', axis=1)
target  =  aux_training['target_col']

In [9]:
X_train, X_val, y_train, y_val = train_test_split(feature, target, test_size=0.2, random_state=0)

print("X_train.shape = ",X_train.shape)
print("X_val.shape = ",X_val.shape)
print("\ny_train.shape = ",y_train.shape)
print("y_val.shape = ",y_val.shape)

    

X_train.shape =  (3966677, 49)
X_val.shape =  (991670, 49)

y_train.shape =  (3966677,)
y_val.shape =  (991670,)


In [10]:
X_train = X_train.sort_index()
y_train = y_train.sort_index()

X_val = X_val.sort_index()
y_val = y_val.sort_index()

In [11]:
# get number of rows of each query (srch_id). Needed to train the model.
qids_train = X_train.groupby("srch_id")["srch_id"].count().to_numpy() 
qids_val   = X_val.groupby("srch_id")["srch_id"].count().to_numpy()


In [ ]:
print("query train ids mean: ",qids_train.mean()) # should we ensure there are enough rows from same srch_id ??? ANS: apparently not
print("query val   ids mean: ",qids_val.mean()) 

In [12]:
FIXED_PARAMS={
              'objective'             : 'lambdarank',
              #'objective'               : 'rank_xendcg',
              'metric'                : 'ndcg',
              'boosting_type'         : 'gbdt',
              'is_unbalance'          : True,
              'num_iterations'        : 300,
              'early_stopping_rounds' : 30}

SEARCH_PARAMS = {'learning_rate': 0.3,
                 'max_depth': 15,
                 'num_leaves': 20,
                 'feature_fraction': 0.8,
                 'subsample': 0.2}

gbm = lgb.LGBMRanker( 
        objective               = FIXED_PARAMS["objective"],
        metric                  = FIXED_PARAMS["metric"],
        boosting_type           = FIXED_PARAMS["boosting_type"],
        is_unbalance            = FIXED_PARAMS["is_unbalance"],
        num_iterations          = FIXED_PARAMS["num_iterations"],
        early_stopping_rounds   = FIXED_PARAMS["early_stopping_rounds"],
        learning_rate           = SEARCH_PARAMS["learning_rate"],
        max_depth               = SEARCH_PARAMS["max_depth"],
        num_leaves              = SEARCH_PARAMS["num_leaves"],
        feature_fraction        = SEARCH_PARAMS["feature_fraction"],
        subsample               = SEARCH_PARAMS["subsample"],
)

# gbm = lgb.LGBMRanker( params )




In [13]:
gbm.fit(X_train, 
        y_train, 
        group=qids_train,
        eval_set=[(X_val, y_val)], 
        eval_group=[qids_val],
        eval_at=[5]
)

C:\Users\Martin\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[1]	valid_0's ndcg@5: 0.92083
[2]	valid_0's ndcg@5: 0.925461
[3]	valid_0's ndcg@5: 0.928891
[4]	valid_0's ndcg@5: 0.930959
[5]	valid_0's ndcg@5: 0.932067
[6]	valid_0's ndcg@5: 0.932803
[7]	valid_0's ndcg@5: 0.933205
[8]	valid_0's ndcg@5: 0.933549
[9]	valid_0's ndcg@5: 0.933744
[10]	valid_0's ndcg@5: 0.934378
[11]	valid_0's ndcg@5: 0.934684
[12]	valid_0's ndcg@5: 0.935156
[13]	valid_0's ndcg@5: 0.935215
[14]	valid_0's ndcg@5: 0.935408
[15]	valid_0's ndcg@5: 0.935507
[16]	valid_0's ndcg@5: 0.935625
[17]	valid_0's ndcg@5: 0.935722
[18]	valid_0's ndcg@5: 0.935854
[19]	valid_0's ndcg@5: 0.935859
[20]	valid_0's ndcg@5: 0.935897
[21]	valid_0's ndcg@5: 0.936002
[22]	valid_0's ndcg@5: 0.93604
[23]	valid_0's ndcg@5: 0.936131
[24]	valid_0's nd

LGBMRanker(early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=True,
           learning_rate=0.3, max_depth=15, metric='ndcg', num_iterations=300,
           num_leaves=20, objective='lambdarank', subsample=0.2)

In [28]:
#del test_data
#gc.collect()

30

In [ ]:
#aux_test = copy.deepcopy(test_data)

In [ ]:
## DONT RUN OR DELETE
#drop columns i dont care or have too many empty values:


aux_test = aux_test.drop('date_time', axis=1)

aux_test = aux_test.drop('comp1_rate', axis=1)
aux_test = aux_test.drop('comp1_inv', axis=1)
aux_test = aux_test.drop('comp1_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp2_rate', axis=1)
aux_test = aux_test.drop('comp2_inv', axis=1)
aux_test = aux_test.drop('comp2_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp3_rate', axis=1)
aux_test = aux_test.drop('comp3_inv', axis=1)
aux_test = aux_test.drop('comp3_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp4_rate', axis=1)
aux_test = aux_test.drop('comp4_inv', axis=1)
aux_test = aux_test.drop('comp4_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp5_rate', axis=1)
aux_test = aux_test.drop('comp5_inv', axis=1)
aux_test = aux_test.drop('comp5_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp6_rate', axis=1)
aux_test = aux_test.drop('comp6_inv', axis=1)
aux_test = aux_test.drop('comp6_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp7_rate', axis=1)
aux_test = aux_test.drop('comp7_inv', axis=1)
aux_test = aux_test.drop('comp7_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp8_rate', axis=1)
aux_test = aux_test.drop('comp8_inv', axis=1)
aux_test = aux_test.drop('comp8_rate_percent_diff', axis=1)

aux_test = aux_test.drop('visitor_hist_starrating', axis=1)
aux_test = aux_test.drop('visitor_hist_adr_usd', axis=1)

aux_test = aux_test.drop('srch_query_affinity_score', axis=1)

#aux_test = aux_test.drop('gross_bookings_usd', axis=1)

aux_test = aux_test.drop('prop_review_score', axis=1)
aux_test = aux_test.drop('prop_location_score2', axis=1)
aux_test = aux_test.drop('orig_destination_distance', axis=1)

In [30]:
test_data = test_data.drop('date_time', axis=1)

In [15]:
# Create prediction over the test set
test_pred = gbm.predict(test_data)

print(test_pred)

[ 0.12118274  0.1589883  -0.00899086 ... -0.53632896  0.16822963
  0.62877632]


In [31]:
# Add new column "predicted_ranking" to the test_set with predicted values of the model
test_data["predicted_ranking"] = test_pred
test_data.head(15)


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,predicted_ranking
0,1,24,216,NaN,NaN,219,3180,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121183
1,1,24,216,NaN,NaN,219,5543,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.158988
2,1,24,216,NaN,NaN,219,14142,2,3.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008991
3,1,24,216,NaN,NaN,219,22393,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298225
4,1,24,216,NaN,NaN,219,24194,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.162342
5,1,24,216,NaN,NaN,219,28181,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.051807
6,1,24,216,NaN,NaN,219,34263,3,4.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.790041
7,1,24,216,NaN,NaN,219,37567,2,4.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.236935
8,1,24,216,NaN,NaN,219,50162,2,3.5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.537600
9,1,24,216,NaN,NaN,219,54937,3,4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.634953


In [32]:
# Sort by srch_id and predicted_ranking
test_data = test_data.sort_values(["srch_id","predicted_ranking"], ascending=[True,False])

In [37]:
test_data.head()

,srch_id,prop_id
9,1,54937
23,1,99484
12,1,61934
4,1,24194
5,1,28181


In [34]:
# Drop all columns but srch_id and prop_id for output format
test_data.drop(test_data.columns.difference(['srch_id','prop_id']), 1, inplace=True)
test_data.head()

,srch_id,prop_id
9,1,54937
23,1,99484
12,1,61934
4,1,24194
5,1,28181
20,1,90385
18,1,82231
6,1,34263
13,1,63894
16,1,74045


In [36]:
# Create output file for competition
test_data.to_csv("out2.csv", index=False)    

# -------------------------------------------------------------------------------------------------------------